# Проект Создание чат-бота Лапина Е.О.
## Бот используется для общения, поиска книг и поиска в энциклопедии

 Данные для книг закачаны с помощью парсера parser_test.ipynb

In [76]:
import torch
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer
import pandas as pd
import numpy as np
import re
import random

# Загрузка предобученной модели для поддержания разговора (sber, на gpt2)

In [77]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", output_attentions=True)

In [78]:
def talking(text):
    inp = text
    encoded = tokenizer.encode(inp, return_tensors="pt")


    result = ru_gpt.generate(encoded,
        max_length=64,
        early_stopping=True,
        no_repeat_ngram_size=5)
    result= tokenizer.decode(result.cpu().flatten().numpy(), skip_special_tokens=True)
    result = result.replace('\n\n—\xa0', '\n')
    result = result.replace('\n\n', '\n')
    result = result.replace('\xa0', '')
    stop_words = ['не согласилась она', 'не согласилась я', 'не согласился он', 'не согласился я', 'согласилась она', 'согласилась я', 'согласился он', 'согласился я','переспросил я','переспросила я','переспросил он', 'переспросила она', 'спросил я', 'спросила я', 'спросила она','спросил он', 'ответил я','ответила я','ответил он', 'ответила она','сказал я', 'сказала я', 'сказала она', 'сказал он', 'повторил я','повторила я', 'повторила она', 'повторил он','удивился я', 'удивилась я', 'удивился он', 'удивилась она','пояснила я', 'пояснил я', 'пояснила она', 'пояснил он',  'кивнула я', 'кивнул я', 'кивнула она', 'кивнул он','подтвердила я', 'подтвердил я','подтвердила она', 'подтвердил он', '.']
    for word in stop_words:
        result = result.replace(word, '')
    result = re.sub('[^а-яА-ЯёЁ0-9?,-=/+\s]', '', result)
    for i in range(20, 2, -1):
        result = result.replace(' '*i, ' ')
    result = result.split('\n')
    try:
        result = result[1]
        
        if result[-1] == ',':
            result = result[:-1]
        result = result.replace('  ', ' ')
        result = result.replace(' Я ', ' я ')
        result = result.replace('- -', '')
        result = result.replace(' я я ', ' я ')
        result = result.replace('-Я', 'Я')
            
        return result
    except:
        return 'Давай поговорим о чем-нибудь другом... копать-хоронить!'

In [79]:
print(talking('ты хочешь есть?'))

Нет, я не голоден


# Загрузка датасета описаний произведений детской литературы с сайта литрес (парсинг)

In [80]:
df_books = pd.read_csv('books_list_litres.csv')
df_books

,Unnamed: 0,title,texts,images,link
0,0,Дикий робот,Удивительная сказка про одного робота по имени...,https://cv7.litres.ru/pub/c/cover_250/39141373...,https://www.litres.ru/book/piter-braun-1753189...
1,1,Пёс по имени Мани,Трогательная история говорящего Лабрадора Мани...,https://cv6.litres.ru/pub/c/cover_250/22649364...,https://www.litres.ru/book/bodo-shefer/pes-po-...
2,2,Хранители мира,Первая часть сказочной дилогии Мелиссы Харрисо...,https://cv6.litres.ru/pub/c/cover_250/67978266...,https://www.litres.ru/book/melissa-harrison/hr...
3,3,Хроники Нарнии. Вся история Нарнии в 7 повестях,Хроники Нарнии – удивительная и прекрасная ист...,https://cv5.litres.ru/pub/c/cover_250/69759154...,https://www.litres.ru/audiobook/klayv-luis/hro...
4,4,"Дело о пропавшей учительнице, или Параллельные...",В жизни 6 «А» происходит нечто фантастическое!...,https://cv8.litres.ru/pub/c/cover_250/66583884...,https://www.litres.ru/book/masha-rupasova/delo...
...,...,...,...,...,...
9598,9598,Девочки с характером. Большая книга романтичес...,«Приз за характер»,https://cv2.litres.ru/pub/c/cover_250/68508527...,https://www.litres.ru/audiobook/vladimir-kukuz...
9599,9599,Сказочные хитрости,Евгений Пермяк – автор многочисленных книг и с...,https://cv0.litres.ru/pub/c/cover_250/68508505...,https://www.litres.ru/book/nina-borisova-32381...
9600,9600,Школа Единорогов. Друзья-первоклассники,Ивка очень ждёт начало учебного года в Школе Е...,https://cv5.litres.ru/pub/c/cover_250/68317952...,https://www.litres.ru/audiobook/anzhelika-rays...
9601,9601,Шантон-Болтон,В сборник вошли добрые и мудрые сказки известн...,https://cv0.litres.ru/pub/c/cover_250/68511209...,https://www.litres.ru/book/anastasiya-sklyanki...


In [81]:
df_books1 = pd.read_csv('books_list_litres1.csv')
df_books1

,Unnamed: 0,title,texts,images,link
0,0,Волшебные краски,"Умные, добрые, философские сказки известного п...",https://cv3.litres.ru/pub/c/cover_250/68511339...,https://www.litres.ru/book/evgeniy-permyak/vol...
1,1,Сказка: Счастливого конца у героя не бывает,Сюжет строится вокруг Ёны и её желания помочь ...,https://cv2.litres.ru/pub/c/cover_250/68564026...,https://www.litres.ru/book/egor-udodov-2906627...
2,2,Хитрый коврик,Книги серии «Читаем сами» необходимы каждому д...,https://cv9.litres.ru/pub/c/cover_250/68511291...,https://www.litres.ru/book/evgeniy-permyak/hit...
3,3,"Сказки Кириной бабушки, или Сказки, что читали...",Жила была Кира. Когда Киры была маленькой и ис...,https://cv8.litres.ru/pub/c/cover_250/68610385...,https://www.litres.ru/book/nikolay-putilin-323...
4,4,Атмосферная поэзия для детей,"Под обложкой этой книги собраны добрые, искрен...",https://cv7.litres.ru/pub/c/cover_250/68610274...,https://www.litres.ru/book/aleksandra-aleksand...
...,...,...,...,...,...
1399,1399,Слепая Вера,Слепая мышка по имени Вера ослушалась мудрого ...,https://cv2.litres.ru/pub/c/cover_250/67775328...,https://www.litres.ru/book/stepan-dmitrievich-...
1400,1400,Все произведения школьной программы в кратком ...,Предлагаемое пособие является универсальным пр...,https://cv5.litres.ru/pub/c/cover_250/67634850...,https://www.litres.ru/book/o-b-marina/vse-proi...
1401,1401,Пингвины-шпионы,Отважный агент 00Ноль (для мамы – Джексон) и е...,https://cv3.litres.ru/pub/c/cover_250/67608830...,https://www.litres.ru/book/sem-hey/pingviny-sh...
1402,1402,Я динозавр,Второклассник Данил большой любитель динозавро...,https://cv8.litres.ru/pub/c/cover_250/67766388...,https://www.litres.ru/book/anastasiya-karp/ya-...


In [82]:
df_books = pd.concat([
    df_books,
    df_books1
])

In [83]:
df_books['title_text'] = df_books['title']+ ' ' + df_books['texts']

In [84]:
df_books['title_text'].head()

0    Дикий робот Удивительная сказка про одного роб...
1    Пёс по имени Мани Трогательная история говорящ...
2    Хранители мира Первая часть сказочной дилогии ...
3    Хроники Нарнии. Вся история Нарнии в 7 повестя...
4    Дело о пропавшей учительнице, или Параллельные...
Name: title_text, dtype: object

In [85]:
df_books.shape

(11007, 6)

In [86]:
df_books = df_books.reset_index()

In [87]:
df_books.columns

Index(['index', 'Unnamed: 0', 'title', 'texts', 'images', 'link',
       'title_text'],
      dtype='object')

In [88]:
df_books = df_books[['title', 'texts', 'images','title_text']]

# Загрузка wikipedia для поиска информации

In [89]:
# !pip3 install transformers torch wikipedia torchvision torchaudio

In [90]:
# !pip3 install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

In [91]:
# !pip install wikipedia

In [92]:
import wikipedia
import re
wikipedia.set_lang("ru")

In [99]:
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:2000]
        # Разбиваем, считая точку разделителем
        wikimas=wikitext.split('.')
        # Отбрасываем всё после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, добавляем ее
                    # к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

In [100]:
# !pip install dialogflow gensim==3.6.0 annoy tqdm stop_words pymorphy2 python-telegram-bot==13.3

In [101]:
# !pip install pandarallel

In [102]:
# !pip3 install gensim==3.6.0 --use-pep517

In [103]:
# !pip3 install dialogflow annoy tqdm stop_words pymorphy2 python-telegram-bot==13.3 pandarallel ipywidgets

In [104]:
# pip freeze

In [105]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
import mmap
import re
import annoy
from tqdm import tqdm_notebook
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import dill
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [106]:
df_books = df_books[['title', 'texts', 'images', 'title_text']].dropna()
df_books.shape

(11004, 4)

### Применим бертовый токенайзер для поиска и обработки запроса на описания книг

In [107]:
bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
bert_tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-

In [108]:
encodings1 = bert_tokenizer(df_books['title_text'].tolist()[:df_books['title_text'].shape[0]//2], 
                      max_length=128, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')
encodings2 = bert_tokenizer(df_books['title_text'].tolist()[df_books['title_text'].shape[0]//2:], 
                      max_length=128, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

In [109]:
tokens = bert_tokenizer('собака и кошка', 
                      max_length=128, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

In [110]:
encodings = np.concatenate([encodings1.input_ids, encodings1.input_ids])
encodings.shape

(11004, 128)

In [111]:
def getbooks_count(text):
    tokens = bert_tokenizer(text, 
                      max_length=128, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')
    intersect_ids_dict = {}
    for i in range(encodings.shape[0]):
        intersect_ids = np.intersect1d(encodings[i], tokens.input_ids, return_indices=False)
        counter = intersect_ids.shape[0]
        intersect_ids_dict[i] = counter
    intersect_ids_dict_sorted = sorted(intersect_ids_dict.items(), key=lambda item: item[1], reverse=True)
    ids_list = []
    
    for el in intersect_ids_dict_sorted[:5]:
        if el[1]>2:
            ids_list.append(el[0])
        else: 
            break
    return ids_list

In [112]:
# Можно добавить еще фразу после ответа бота. Пока не будем этого делать
# extra_list = 'Звонило прошлое, просилось в будущее…. Злость — это такое состояние, когда язык и руки работают быстрее мозга. Ошибочно набранный номер телефона никогда не бывает занят. Можно не бояться быть дураком. Кругом все свои. Господа, за вами будущее. Отойдите. Мне чужого не надо, но свое я возьму — чьё бы оно ни было!. Хватит париться, живи как нравится!. Хватит сидеть без денег!. Будешь много знать – очень расстроишься. Если ты сегодня не позвонишь, звонить буду я, причём не тебе…. Все к лучшему. Хотя, конечно, хрен его знает. Если ты умнее всех — кто ж это оценит?. Твоя голова всегда в ответе за то, куда сядет твой зад. Меня деньги не волнуют! Они меня успокаивают. У китайцев нет слова «изменил», а есть слово «перепутал»…. Жизнь грустная… Зато зарплата смешная. Шоколад вдвойне вкусней, если его нельзя. В аптеках надо продавать время, оно же лечит. Все хотят попасть в рай, но никто не хочет умирать. Если точно знаешь, кто виноват – не выдавай себя. Душа в душу могут жить только матрешки. Будильник ненавидят всего в двух случаях – когда он звенит, и когда он не прозвенел. Вместе мы горы свернём! Ну, или дров наломаем…. Воздух не чувствуется пока его не испортят. Странно, что слово «трудновыговариваемо» настолько трудновыговариваемо. В жизни каждого человека наступает момент, когда давно уже пора спать. Если вы чувствуете, что охрипли — возможно, кто-то оглох. Человек на вершине горы туда не упал. Не важно, что вы в дом берете котенка или мужика! Полгода миленькая моська, а дальше хитрый наглый тип!. Все инструкции на русском языке должны начинаться со слов: «Ну что, придурок, уже сломал?». Если девушка просит тебя оставить её одну, оставь её одну, только ни в коем случае не оставляй её одну!. Короче, удачи тебе, братишка. Сегодня лег спать в 22-00, уже 3-10, а я все еще не могу нарадоваться тому, что лег так рано!. Людям надо доверять, не деньги, конечно, или секреты, а так — вообще. Труднее всего выполнить обещание, данное себе самому. Знаешь ведь: если что — тебя поймут и простят. «Нельзя» бывает только для тех, кто спрашивает. Кто не рискует, тот пьёт водку на поминках того, кто рисковал. Когда я был маленьким, то очень хотел вырасти, кто же знал, что здесь такая подстава. Если можно, а тем более нужно, то как-то не очень и хочется. Лечебные свойства хрена: если его положить на что-нибудь, то сразу становится легче. Мудрость — это возрастное замедление работы мозга, приводящее к невозможности принятия скоропалительных решений. Повесила оберег над дверью, от всякой нечисти, вышла в магазин, вернулась, ключ в двери сломался, зайти не могу. А что если Ленин лежит в мавзолее потому, что его заколдовала злая фея, и если его поцеловать, то заклятие спадет и СССР вернется?. Те, кто рассказывает вам обо мне, рассказывают мне о вас. Никогда не забывайте об этом. Все понтуются, абсолютно все, а те, кто не понтуется, потом понтуется тем, что не понтуется. При пожаре соблюдайте очередность! Сначала — выйти из здания, потом — написать в соц сетях: VK, Одноклассниках, Фейсбуке, Твиттере. Совесть — она такая, она мучает не тех, кого должна мучить, а тех, у кого она есть. У нас на работе стоит кофейный автомат, я уже проиграл в него зарплату. Петух увидел курицу в микроволновке и говорит: «Ядрен батон! в деревне яйца нести некому, а она тут на каруселях катается!». Хвост у человека отпал, а необходимость им вилять осталась. Занимать деньги нужно у пессимистов, они заранее знают, что им не отдадут. Сидя по уши в дерьме, рта особенно не раскроешь. Органы как органы и лишь задница массовик затейник. История Мальвины доказала, что женщина легко может полюбить мужчину пусть и с деревянной головой, но зато обладающего золотым ключиком. Идешь такой с кружкой чая в руках и книгой, и вместо книги кидаешь на кровать чай. Раньше, когда у меня не было денег, я связывал это с отсутствием работы, сейчас у меня есть работа, но, видимо, я что-то делаю не так. Секрет моего всегда прекрасного самочувствия и постоянно хорошего настроения? Вегетарианство, йога и немного мяса и водочки на ужин!. Люблю выходные! Можно быстро и без пробок доехать на работу. Завтра — мифическое место, где ты бегаешь по утрам, живешь полной жизнью и решаешь свои жизненные проблемы. Прогресс! Умные очки, умные часы, умные кроссовки, умные микроволновки! Одни люди тупые остались. Вы замечали, что если ты в купе заходишь последним, то у тебя такое ощущение, как будто ты заходишь в гости?. Проснулся пораньше, чтобы подольше походить с недовольным лицом. Человек рождён для счастья, а не для того, “чтобы слушаться”!. Лучше быть последним — первым, чем первым — последним. На случай, если буду нужен, то я там же, где и был, когда был не нужен. Если волк молчит, то лучше его не перебивай. Легко вставать, когда ты не ложился. Кем бы ты ни был, кем бы ты не стал, помни, где ты был и кем ты стал. Иногда жизнь — это жизнь, а ты в ней иногда. В жизни полно лжи и грязи, она не так красива. Даже твой лучший друг может не поделиться пивом. Бесплатный сыр бывает только бесплатным. Я не боюсь ударов в спину, гораздо страшнее, если это моя спина. Сначала потом, затем, снова опять. Если в дверь не постучаться, ее никогда не откроют. Штаны коту не полагаются, мессир. Уж не прикажете ли Вы мне надеть и сапоги? Кот в сапогах бывает только в сказках, мессир. Но видели ли Вы когда-нибудь кого-нибудь на балу без галстука? Я не намерен оказаться в комическом положении и рисковать тем, что меня вытолкают в шею!. — Голову ему оторвать! — сказал кто-то сурово на галерке. Tаити, Tаити! Не были мы ни в какой Таити! Нас и здесь неплохо кормят. Свободу попугаям! Сво-бо-ду по-пу-га-ям!. Вернись, я всё прощу!. А с чем пироги-то?. У тебя — своя сказка, а у меня — своя. Ой бяда, бяда! Разорение! Запасы не меряны! Убытки не считаны! Разоримся по миру пойдем. Это что, сказка такая? - Это жизня такая. Говорят, вам счастья привалило? - Бессо-овестно врут!. Я не жадный, я домовитый!. Счастье - это когда у тебя все дома. Я птица вольная! Куда хочу - туда лечу!. Нафаня, сундук украли!. Кузенька! Вернись, яхонтовый мой!. Не очень-то вежливо уходить из гостей сразу, как только ты наелся. Никто не может грустить, когда у него есть воздушный шарик!. И это всё потому что у кого-то слишком узкие двери. Всё потому что кто-то слишком много ест!. Я думал, думал и наконец все понял! Это неправильные пчелы!. Кто ходит в гости по утрам, тот поступает мудро!. Кажется, дождь собирается. Я тучка, тучка, тучка, я вовсе не медведь. Безвозмездно — то есть даром. Поздравляю с днём рожденья, желаю счастья в личной жизни, Пух!. Копать-хоронить!!!.  А где дедушка-то спит? Да вон там в коридоре, на коврике, а если не слушается, так я его веником!. Человек собаке друг - это знают все вокруг!. Или, может, обратиться к врачу?. Ни-че-го я не хо-чу!. Фокусы — это ловкость рук и никакого мошенства. Щас спою!. Ты заходи, если что!. А мы тут, знаете, все плюшками балуемся. Пустяки, дело-то житейское!. Спокойствие, только спокойствие!. Поверь мне, не в пирогах счастье. Он улетел, но обещал вернуться. Я мужчина хоть куда! В полном расцвете сил. Это я шалю, ну, то есть балуюсь. Там ещё осталось немножечко варенья?. А как же я? Ведь я же лучше? Лучше собаки? А?. О, я самый больной в мире человек. Ну я так не играю. А у вас молоко убежало. Ребята, давайте жить дружно!. Выходи, подлый трус!. У меня есть мысль, и я её думаю!. Не будем говорить - кто, хотя это был слоненок. Я не хочу быть, извините, напримером. А в попугаях–то я гора-а-аздо длиннее!. Я про одно и то же два раза думать не умею!. А я такой голодный, как айсберг в океане! А ты такой прекрасный, как летом эскимо. Начинаем утреннюю зарядку для тех, кто нас смотрит вечером. Лучший мой подарочек - это ты!. Не правильно ты, дядя Федор, бутерброд ешь! Ты его колбасой кверху держишь, а надо колбасой на язык класть! Так вкуснее получится. Это я, почтальон Печкин! Принес заметку про вашего мальчика!. С ума поодиночке сходят, это только гриппом все вместе болеют. До чего же бывают люди до чужого добра жадные. В такую погоду свои дома сидят, телевизор смотрют! Только чужие шастают! Не будем дверь открывать!. Не надо меня из ружья щелкать! Я может, только жить начинаю — на пенсию перехожу. Поздравляю тебя, Шарик, ты балбес!. Усы, лапы и хвост — вот мои документы!. Это индейская национальная народная изба — «фиг вам» называется. Ничего себе! Вашу маму и там, и тут передают! До чего техника дошла!. Мы строили, строили и наконец построили! Ура!. Нет, мы не хотим в живой уголок — мы хотим в пионеры!. Чебурашка, Чебурашка, где ты?. Ведь кто же, кроме тебя, звезды-то считать будет?!. Все-таки хорошо, что мы снова вместе. Казнить нельзя помиловать. Его что, трамвай переехал?. У него получилось полтора землекопа. Птица Говорун отличается умом и сообразительностью. Я же просил 400 капель! А здесь 402. Ах, если бы сбылась моя мечта, какая жизнь настала бы тогда. А я не хочу, не хочу по расчету, а я по любви, по любви хочу. Эх, жизнь моя, жестянка! Да ну ее в болото! Живу я как поганка, а мне летать, а мне летать, а мне летать охота!. Шла лесною стороной, увязался черт за мной! Думала: мужчина - что за чертовщина?!. Я там и тут — куда пошлют, а посылают часто. Как вы яхту назовёте, так она и поплывет. Ха-ха-ха! Руки вверх! Ваша песенка спета. Не ходи туда, там тебя ждут неприятности. А давай вместе бояться, а?'
# extra_list = extra_list.split('. ')
# extra_list

In [113]:
# len(extra_list)

In [114]:
# rand_extra = extra_list[random.randint(0,len(extra_list)-1)]
# rand_extra 

In [115]:
# Создание бота в телеграмм
# @botfather
# /start
# /newbot - create a new bot
# name1
# name1_BOT
#. ->. API

In [ ]:
# заменить на свой токен
updater = Updater("6501041402:AAG6_eNolYfOz0FpRGJiD35DHVGGxwR32bE", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся! Я умею искать книги (для этого напиши "почитать" и то, о чем тебе было бы интересно), я порекомендую книги. Могу находить полезную информацию в энциклопедии (для этого напиши "найди" и то, что тебя интересует). Могу просто поболтать или сочинить что-нибудь в ответ на твои слова.')

def textMessage(update, context):
    
    input_txt = update.message.text
    if input_txt.lower() == "как тебя зовут" or input_txt.lower() == "как тебя зовут?":
        context.bot.send_message(chat_id=update.message.chat_id, text="Жан-Жак")
        return
    if input_txt.lower() == "привет" or input_txt.lower() == "здравствуй":
        context.bot.send_message(chat_id=update.message.chat_id, text="Привет!")
        return
    # поиск книг
    if "почитать" in input_txt.lower():
        text = input_txt.lower().replace("почитать", '')
        ids = getbooks_count(text)
        if ids != []:
            for el in ids:
                title, image, text = df_books['title'][el], df_books['images'][el], df_books['texts'][el]
                context.bot.send_message(chat_id=update.message.chat_id, text="Название книги: {} Обложка: {} Описание {}".format(title, image, text))
            return

        else: 
            context.bot.send_message(chat_id=update.message.chat_id, text="Я не могу найти такие книги в моей библиотеке... Копать-хоронить!")
            return
    # поиск в энциклопедии
    elif "найди" in input_txt.lower():
        text = input_txt.lower().replace("найди", '')
        result = getwiki(text)
        context.bot.send_message(chat_id=update.message.chat_id, text=result)
        return
     # разговор
    else:
        
        text = input_txt
        #result = talking(text) + '\n' + extra_list[random.randint(0,len(extra_list)-1)]
        result = talking(text)
        context.bot.send_message(chat_id=update.message.chat_id, text=result)
        return
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/home/ekaterina/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
# Имя бота JacquesTalkbot